# Naive Machine Learning Models for WSB Stock Prediction

I'll be trying to use decision tree and random forest regressors to predict the stock of Tesla (TSLA) stock. I anticipate that my results won't be as good as the LSTM model, but I wanted to see how the models we learned in class would compare to it. 

In [144]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import pickle

import yfinance as yf

import datetime

In [2]:
def min_max_date(df):
    """
    Get the min and max date of the ticker being mentioned in our WSB dataset
    """
    min_date = df.sort_values('datetime').iloc[0]["datetime"].date()
    max_date = df.sort_values('datetime', ascending= False).iloc[0]["datetime"].date()
    return(min_date, max_date)

def explode_action_column(df):
    """
    Explodes columns of
    dict(puts: [., ., .,]
         calls: [., ., .,]
         sell: [., ., .,]
         buy: [., ., .,])
         
    into four columns ...
    |puts|calls|sell|buy|
    """
    return pd.concat([df.drop(['sentiment'], axis=1), df['sentiment'].apply(pd.Series)], axis=1)

def date_from_datetime(df):
    """
    Make datetime to date
    """
    df_copy = df.copy()
    df_copy["Date"] = df_copy["datetime"].dt.date
    return df_copy

def flatten_filter(df, ticker, filter = True):
    """
    Explodes the column of lists of tickers into multiple rows and then filter for ticker
    """
    df_copy = df.copy()
    df_flatten = df_copy.explode('tickers')
    if filter:
        df_flatten = df_flatten[df_flatten['tickers'] == ticker]
    else:
        return df_flatten
    return df_flatten

def indicator_actions(df, ticker):
    """
    Make indicator variables for ticker instead of list of tickers
    """
    df_copy = df.copy()
    df_copy = explode_action_column(df_copy)
    df_copy["puts"] = ((df_copy["puts"].apply(len) != 0) & 
                      (df_copy["puts"].astype('str').str.contains(ticker))).astype(int)
    df_copy["calls"] = ((df_copy["calls"].apply(len) != 0) & 
                       (df_copy["calls"].astype('str').str.contains(ticker))).astype(int)
    df_copy["buy"] = ((df_copy["buy"].apply(len) != 0) & 
                     (df_copy["buy"].astype('str').str.contains(ticker))).astype(int)
    df_copy["sell"] = ((df_copy["sell"].apply(len) != 0) & 
                      (df_copy["sell"].astype('str').str.contains(ticker))).astype(int)
    return df_copy

def plot_actions(df, ticker, min_date, max_date, hide_score = True):
    """
    Plot the aggregate sum of sell, buy, calls ,puts for each day within range for a specific ticker
    """
    df_copy = df.copy()
    
    # Aggregate financial action frequency per day
    if hide_score:
        agg_sell = df_copy[['Date','sell','buy','calls','puts']].groupby('Date').agg('sum')
    else:
        agg_sell = df_copy[['Date','sell','buy','calls','puts','score','ups']].groupby('Date').agg('sum')


    # Filter for range specified
    mask = (agg_sell.index > pd.to_datetime("2018-01-01").date()) & (agg_sell.index <= max_date)

    agg_sell.loc[mask].plot(figsize=(12, 10), linewidth=2.5)
    plt.xlabel("Date", labelpad=15)
    plt.ylabel("Movement Mentions", labelpad=15)
    plt.title("Movement mentions for {0} from {1} to {2}".format(ticker, str(min_date), str(max_date)), y=1.02, fontsize=22);

def plot_stock_vs_wsb(df,ticker, min_date, max_date, stock_col, action_col):
    """
    Plot the a specfic financial action mention in WSB comments vs historical data
    """
    df_copy = df.copy()
    
    # Aggregate financial action frequency per day
    agg_sell = df_copy[['Date','sell','buy','calls','puts','score','ups']].groupby('Date').agg('sum')
    
    
    # Get the minimum date of that ticker mentioned on our WSB comment
    MIN_DATE, MAX_DATE = min_max_date(df)

    # Pull the data from yahoo finance api
    stock_data = yf.download(ticker, start = MIN_DATE, end = MAX_DATE)
    
    # Filter for range specified
    mask = (stock_data.index > np.datetime64(str(min_date))) & (stock_data.index <= np.datetime64(max_date))

    plot_multi(stock_data.loc[mask].join(other = agg_sell)[[stock_col,action_col]], figsize=(10,5))

    plt.xlabel("Date", labelpad=15)
    plt.ylabel("Daily {0} mentions".format(action_col), labelpad=15)
    plt.title("Daily {1} mentions vs {2} from {3} to {4} for {0}".format(ticker, action_col, stock_col, str(min_date), str(max_date)), y=1.02, fontsize=22);

In [3]:
data = pickle.load(open("wsb.pkl", "rb"))

In [4]:
tesla_df = date_from_datetime(flatten_filter(data, 'TSLA'))

In [110]:
ticker = "TSLA"

# Get the minimum date of that ticker mentioned on our WSB comment
MIN_DATE, MAX_DATE = min_max_date(tesla_df)

# add 5 days to the max date because we will be aggregating da
MAX_DATE += datetime.timedelta(days=5)

# Pull the data from yahoo finance api
stock_data = yf.download(ticker, start = MIN_DATE, end = MAX_DATE)

[*********************100%***********************]  1 of 1 completed


In [86]:
stock_data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2013-04-09,41.799999,41.830002,40.330002,40.500000,40.500000,1696100
2013-04-10,40.700001,42.009998,40.610001,41.860001,41.860001,2121100
2013-04-11,42.060001,44.549999,41.750000,43.590000,43.590000,3447400
2013-04-12,43.250000,45.139999,43.049999,43.750000,43.750000,3149400
2013-04-15,43.500000,43.799999,42.509998,43.299999,43.299999,1681400


In [124]:
tesla_sent = indicator_actions(tesla_df, 'TSLA')
tesla_sent["body"] = tesla_sent["body"].str.replace('\\n','')

'Lots of big companies dipped today. Held onto MSFT and AMZN to grab some options on TSLA and GOOG instead. Looking to grab NFLX too if prices continue staying low.'

In [9]:
agg_sell = tesla_sent[['Date','sell','buy','calls','puts','score']].groupby('Date').agg('sum')
agg_sell.head()

,sell,buy,calls,puts,score
Date,,,,,
2013-04-09,0,1,0,0,1
2013-05-08,0,1,0,0,2
2013-05-09,1,0,0,0,1
2013-05-10,0,2,0,0,3
2013-05-12,0,1,0,0,1


In [10]:
df = agg_sell.join(stock_data).dropna().reset_index()

In [11]:
# Sort DataFrame by date
df = df.sort_values('Date')

# Double check the result
df.head()

,Date,sell,buy,calls,puts,score,Open,High,Low,Close,Adj Close,Volume
0,2013-04-09,0,1,0,0,1,41.799999,41.830002,40.330002,40.500000,40.500000,1696100.0
1,2013-05-08,0,1,0,0,2,57.500000,58.200001,55.709999,55.790001,55.790001,6769900.0
2,2013-05-09,1,0,0,0,1,70.120003,75.769997,63.689999,69.400002,69.400002,28605000.0
3,2013-05-10,0,2,0,0,3,69.650002,81.000000,69.250000,76.760002,76.760002,25082600.0
4,2013-05-14,1,2,0,0,3,94.220001,97.120003,81.150002,83.239998,83.239998,37163900.0


In [26]:
df["mid_price"] = (df["High"] + df["Low"])/2

In [27]:
df.head()

,Date,sell,buy,calls,puts,score,Open,High,Low,Close,Adj Close,Volume,mid_price
0,2013-04-09,0,1,0,0,1,41.799999,41.830002,40.330002,40.500000,40.500000,1696100.0,41.080002
1,2013-05-08,0,1,0,0,2,57.500000,58.200001,55.709999,55.790001,55.790001,6769900.0,56.955000
2,2013-05-09,1,0,0,0,1,70.120003,75.769997,63.689999,69.400002,69.400002,28605000.0,69.729998
3,2013-05-10,0,2,0,0,3,69.650002,81.000000,69.250000,76.760002,76.760002,25082600.0,75.125000
4,2013-05-14,1,2,0,0,3,94.220001,97.120003,81.150002,83.239998,83.239998,37163900.0,89.135002


In [14]:
# First calculate the mid prices from the highest and lowest
high_prices = np.matrix(df.loc[:,'High'])
low_prices = np.matrix(df.loc[:,'Low'])
mid_prices = (high_prices+low_prices)/2.0

In [15]:
train_data = mid_prices[0,:800]
test_data = mid_prices[0,800:]

In [16]:
# Scale the data to be between 0 and 1
# When scaling remember! You normalize both test and train data with respect to training data
# Because you are not supposed to have access to test data
scaler = MinMaxScaler()

In [17]:
train_data = train_data.reshape(-1,1)
test_data = test_data.reshape(-1,1)
test_data

matrix([[309.82501221],
        [321.78999329],
        [320.81500244],
        [328.58000183],
        [346.48500061],
        [343.5       ],
        [352.67500305],
        [357.96000671],
        [364.11500549],
        [358.125     ],
        [358.19000244],
        [356.23999023],
        [342.125     ],
        [332.98500061],
        [334.67498779],
        [345.14500427],
        [351.56498718],
        [348.13499451],
        [347.31500244],
        [321.08999634],
        [305.30500793],
        [307.03500366],
        [313.25999451],
        [323.44000244],
        [318.50500488],
        [318.        ],
        [314.41499329],
        [310.70500183],
        [316.61999512],
        [320.875     ],
        [318.77500916],
        [317.47499084],
        [299.17999268],
        [300.13499451],
        [302.05999756],
        [307.17001343],
        [301.51499939],
        [291.11500549],
        [293.69500732],
        [298.        ],
        [336.57499695],
        [348.764

In [18]:
# Train the Scaler with training data and smooth data
smoothing_window_size = 10
for di in range(0,100,smoothing_window_size):
    scaler.fit(train_data[di:di+smoothing_window_size,:])
    train_data[di:di+smoothing_window_size,:] = scaler.transform(train_data[di:di+smoothing_window_size,:])

# You normalize the last bit of remaining data
scaler.fit(train_data[di+smoothing_window_size:,:])
train_data[di+smoothing_window_size:,:] = scaler.transform(train_data[di+smoothing_window_size:,:])

Diverges from here

In [123]:
tesla_df2 = tesla_sent[['body','Date','sell','buy','calls','puts','score']].set_index('Date')
tesla_df2.head()

,body,sell,buy,calls,puts,score
Date,,,,,,
2014-08-07,TSLA is trading on news. Everytime some bad ne...,0,1,0,0,1
2014-04-26,Lots of big companies dipped today. Held onto ...,0,1,0,0,1
2014-05-08,Unless TSLA pulls a TRIP and rallies back to g...,0,1,0,0,1
2014-02-26,If you have TSLA stock your already making mon...,0,1,0,0,0
2014-05-08,&gt; Trash of a stock.Hardly. They're doing a ...,0,1,0,0,2


In [125]:
for row in range(len(stock_data)):
    # for each day, calculate the average mid price ((high + low) / 2) for the next 3 days 
    
    stock_data.loc[stock_data.index[row], 'mid_price_next_3_days'] = \
    ((stock_data.iloc[row+1:row+4,1] + stock_data.iloc[row+1:row+4,2])/2).mean()
    
# the last date of stock prices will be NA because there is no next day mid price to average
stock_data = stock_data.dropna()

# we only want the mid price (to predict)
stock_mid_data = stock_data["mid_price_next_3_days"]

In [127]:
tesla_stock_df = tesla_df2.join(stock_mid_data)
tesla_stock_df.head()

,body,sell,buy,calls,puts,score,mid_price_next_3_days
Date,,,,,,,
2013-04-09,"TSLA, SSYS, SCTY, OHI, RVBD",0,1,0,0,1,42.851666
2013-05-08,"*Zing!*JROD teach me your ways. I am noob, the...",0,1,0,0,2,76.143333
2013-05-09,I like the move esp in the short run although ...,1,0,0,0,1,82.611668
2013-05-10,I actually did play a oom strangle on TSLA bef...,0,1,0,0,1,85.068334
2013-05-10,I'm very curious what you paid here. On my bid...,0,1,0,0,2,85.068334


In [120]:
tesla_stock_df["body"].iloc[1]

'*Zing!*\n\nJROD teach me your ways. I am noob, the most beholden of noobs, but I came into a chunk early april and have built it up 16% in the last month. (YES GOD DAMNIT, MOST OF IT WAS TESLA...OK OK, AND THE TESLA JUMP AFTER EARNINGS TODAY). I would say 8-9% growth from TSLA and the others from random hunches that mercifully played off.\n\nANYHOO, convince me that technical analysis is extremely valuable. Where can I learn more about it? I want to be a wizard.'